In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import os
import sys
from spaces import *
import gtda
import matplotlib.pyplot as plt
from pathlib import Path #auxilliary module
from gtda.plotting import plot_diagram
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram
from gtda.diagrams import PersistenceEntropy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [17]:
sys.path.append("C:\\Users\\Joe\\Documents\\University\\4th Year\\symmetric_tda")

In [2]:
def extract_key_vectors(diagrams, n):
    key_vectors = [None] * 4

    for batch in diagrams:
        for vector in batch:
            V1, V2, V3 = vector # v1 = birth, v2 = death, v3 = dimension
            
            # First key vector: V[0] is the vector with the latest death for which v3 = n - 1
            if V3 == n - 1:
                if key_vectors[0] is None or V2 > key_vectors[0][1]:
                    key_vectors[0] = vector

            # Second key vector: V[1] is the vector with the earliest birth for which v3 = n
            if V3 == n:
                if key_vectors[1] is None or V1 < key_vectors[1][0]:
                    key_vectors[1] = vector


            # Third key vector: V[2] is the vector with the latest death for which v3 = n
            if V3 == n:
                if key_vectors[2] is None or V2 > key_vectors[2][1]:
                    key_vectors[2] = vector

            # Fourth key vector: V[1] is the vector with the earliest birth for which v3 = n + 1
            if V3 == n + 1:
                if key_vectors[3] is None or V1 < key_vectors[3][0]:
                    key_vectors[3] = vector

    return key_vectors

In [3]:

class Torus(Space): #regular torus
    def __init__(self, R, r):
        """
        Create a torus with major radius R and minor radius r.
        The embedding dimension is 3 (in 3D space).
        """
        self.R = R  # Major radius
        self.r = r  # Minor radius
        super().__init__('Torus', 3, 2)

    def sample(self, sample_size, num_samples=1):
        """
        Returns a numpy matrix of shape n x edim, where each row is a
        random point on the torus. To do this, we generate random angles
        theta and phi, and then compute the corresponding points on the torus.
        """
        theta = np.random.uniform(0, 2 * np.pi, sample_size * num_samples)
        phi = np.random.uniform(0, 2 * np.pi, sample_size * num_samples)

        x = (self.R + self.r * np.cos(phi)) * np.cos(theta)
        y = (self.R + self.r * np.cos(phi)) * np.sin(theta)
        z = self.r * np.sin(phi)

        points = np.vstack((x, y, z)).T
        self.points = points.reshape(num_samples, sample_size, self.edim)
        return self.points

In [4]:
class NTorus(Space):
    def __init__(self, n):
        """
        Create an n-dimensional torus. The embedding dimension is 2n.
        """
        self.n = n
        super().__init__('T^{' + str(n) + '}', 2 * n, n)

    def sample(self, sample_size, num_samples=1):
        """
        Returns a numpy matrix of shape (num_samples, sample_size, 2n), where each row is a
        random point on the n-dimensional torus. To do this, we generate random angles
        theta_i uniformly distributed between 0 and 2*pi for each dimension.
        """
        theta = np.random.uniform(0, 2 * np.pi, (sample_size * num_samples, self.n))

        # Compute the corresponding points in the embedding space
        points = np.hstack((np.cos(theta), np.sin(theta)))

        self.points = points.reshape(num_samples, sample_size, self.edim)
        return self.points

In [5]:
T = Space.fit(NTorus(3), 70, 1)

In [ ]:
results = []
#plot_diagram(S[0])
k = 55 # number of points sampled

for n in range(2, 6):

    V = extract_key_vectors(Space.fit(Sphere(n), k, 1), n)

    # Calculate the difference in distance between the birth and death of key vector 1 and key vector 0, respectively
    differencebefore = V[1][0] - V[0][1]
    # Calculate the difference in distance between the birth and death of key vector 3 and key vector 2, respectively
    differenceafter = V[3][0] - V[2][1]
    # Append the result to the list
    results.append({'Dimension of Sphere': n, 'Difference before': differencebefore, 'Difference after': differenceafter})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results
print(results_df)

   Dimension of Sphere  Difference before  Difference after
0                    2           0.150142          0.000032
1                    3           0.019721          0.009190
2                    4           0.162517          0.043178
3                    5           0.027065          0.000331
